# Homework

TODO: Leírást gyártani

In [ ]:
!pip install -r requirements.txt -q

# pip didn't find pyg-lib for some reason
#!pip install pyg-lib>=0.4 torch-scatter>=2.1 torch-sparse>=0.6 torch-cluster>=1.6 torch-spline_conv>=1.2 -f https://data.pyg.org/whl/torch-2.5.0+cu121.html --force-reinstall


In [ ]:
# Ego networks
!wget -P /Data https://snap.stanford.edu/data/twitter.tar.gz
!unzip -qq /Data/twitter.tar.gz
!rm twitter.tar.gz

# Edge list
!wget -P /Data https://snap.stanford.edu/data/twitter_combined.txt.gz
!unzip /Data/twitter_combined.txt.gz
!rm twitter_combined.txt.gz


In [12]:
# General
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display

#Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Graph
import networkx as nx
import torch_geometric as pyg	# This import doesn't work
import torch_geometric.nn as gnn
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.sampler import NegativeSampling
from torch_geometric.utils.convert import from_networkx

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)

# Default device breaks node and edge sampling
#torch.set_default_device(DEVICE)

print(f"Training on {DEVICE} in PyTorch version {torch.__version__}")


Training on cuda:0 in PyTorch version 2.5.1+cu121


## Data loading and cleaning

### Data preparation

In [3]:
# Get nodeID's with associated files (ego nodes)
nodes = []
for file in os.listdir("./Data/twitter"):
	if file.split(".")[0] not in nodes:
		nodes.append(file.split(".")[0])
nodes[:5]


['100318079', '10146102', '101859065', '101903164', '102765423']

In [ ]:
# All feat names in order of discovery
allfeatnames = pd.Series(dtype=str)
# Node features as dictionaries of followed #-s and @-s (with multiplicities)
allfeats = pd.DataFrame(columns=["Features"])
c = 0

for node in nodes:
	c+=1
	if not c%10:
		print(f"File {c} out of {len(nodes)}")

	# Remove duplicates with special characters and lowercase-uppercase shenanigans
	featnames = pd.read_csv(f"./Data/twitter/{node}.featnames", sep=" ", header=None)[1].str.rstrip(".,;:'+-!?)(}{][").str.lstrip("#@").str.lower()
	allfeatnames = pd.Series(pd.concat([allfeatnames, featnames]).unique())

	# Feature vectors except ego node
	featdf = pd.read_csv(f"./Data/twitter/{node}.feat", sep=" ", header=None).set_index(0).set_axis(featnames, axis=1)

	# Feature vector of ego node
	egofeat = pd.read_csv(f"./Data/twitter/{node}.egofeat", sep=" ", header=None).set_axis(featnames, axis=1)
	egofeat = pd.concat([pd.DataFrame([int(node)], columns=[0]), egofeat], axis=1).set_index(0)

	# All feature vectors in a DataFrame
	featdf = pd.concat([egofeat, featdf])
	featdf = featdf.T.groupby(level=0).sum().T

	# Create bag-of-words style dictionaries to later turn into tensors
	featdf["Features"] = [{col: featdf.at[idx, col] for col in featdf.columns if featdf.at[idx, col]} for idx in featdf.index]
	
	# Concatenate
	allfeats = pd.concat([allfeats, featdf.Features])

	# Edges - use twitter_combined instead
	# edgedf = pd.read_csv(f"./Data/twitter/{node}.edges", sep=" ", header=None).T
	
# Drop the (one) empty string and save
allfeatnames = allfeatnames.dropna()
allfeatnames.to_json("feature_names.json")
print(allfeatnames)

# Drop the empty dicts (there's over 10.000 unique nodes following nothing)
allfeats = allfeats.loc[allfeats["Features"] != {}].sort_index()
allfeats


File 10 out of 973
File 20 out of 973
File 30 out of 973
File 40 out of 973
File 50 out of 973
File 60 out of 973
File 70 out of 973
File 80 out of 973
File 90 out of 973
File 100 out of 973
File 110 out of 973
File 120 out of 973
File 130 out of 973
File 140 out of 973
File 150 out of 973
File 160 out of 973
File 170 out of 973
File 180 out of 973
File 190 out of 973
File 200 out of 973
File 210 out of 973
File 220 out of 973
File 230 out of 973
File 240 out of 973
File 250 out of 973
File 260 out of 973
File 270 out of 973
File 280 out of 973
File 290 out of 973
File 300 out of 973
File 310 out of 973
File 320 out of 973
File 330 out of 973
File 340 out of 973
File 350 out of 973
File 360 out of 973
File 370 out of 973
File 380 out of 973
File 390 out of 973
File 400 out of 973
File 410 out of 973
File 420 out of 973
File 430 out of 973
File 440 out of 973
File 450 out of 973
File 460 out of 973
File 470 out of 973
File 480 out of 973
File 490 out of 973
File 500 out of 973
File 510 

,Features
12,"{'claychristensen': 1, 'coachella': 1, 'gabrie..."
12,"{'charlieroseshow': 1, 'claychristensen': 1, '..."
12,"{'claychristensen': 1, 'jack': 1, 'makeitwork'..."
12,"{'jack': 1, 'jbrewer': 1, 'twitter': 1, 'twitt..."
12,"{'claychristensen': 1, 'foodtruck': 1, 'gzahnd..."
...,...
568552194,"{'a_olivieri11': 1, 'atlanta_falcons': 1, 'dak..."
568627575,"{'lucasgomezbr': 1, 'natystadler': 1}"
568655523,"{'dailyprofet': 1, 'paixaojovem': 1}"
568699879,"{'hotmail.com': 1, 'lualone': 1, 'mileybrcom':..."


In [22]:
# allfeats contains many duplicate indices which we need to deal with

# Function to aggregate the dicts of duplicate indices, taking the maximum of all multiplicities
# (The feature vectors are different across files)
def dict_aggregation(dicts):
	out = {}
	for d in dicts:
		for k,v in d.items():
			if k in out.keys() and out[k] >= v:
				pass
			else:
				out[k] = v
	return out

# Update node dicts with duplicate indices, then remove unnecessary duplicates
for idx in allfeats[allfeats.index.duplicated()].index.unique():
	agg = dict_aggregation(allfeats.loc[idx].Features)
	allfeats.loc[idx, "Features"] = [agg] * len(allfeats.loc[idx])

allfeats = allfeats[~allfeats.index.duplicated()]
allfeats


,Features,Bag_of_Words
12,"{'claychristensen': 1, 'coachella': 1, 'gabrie...",12
13,"{'brainpicker': 1, 'ev': 1, 'eventbrite': 1, '...",13
17,"{'amyquispe': 1, 'baratunde': 1, 'busterbenson...",17
20,"{'aaronsw': 1, 'abdur': 1, 'amac': 1, 'dustin'...",20
47,"{'al3x': 1, 'alexandrak': 1, 'allspaw': 1, 'av...",47
...,...,...
568552194,"{'a_olivieri11': 1, 'atlanta_falcons': 1, 'dak...",568552194
568627575,"{'lucasgomezbr': 1, 'natystadler': 1}",568627575
568655523,"{'dailyprofet': 1, 'paixaojovem': 1}",568655523
568699879,"{'hotmail.com': 1, 'lualone': 1, 'mileybrcom':...",568699879


In [ ]:
# Function to create a Bag of Words tensor given an index
def id2bow(index, bag_size = 195):
	bow = torch.zeros(bag_size, dtype=torch.int64)
	dictionary = allfeats.loc[index, "Features"]
	j = 0
	for key,value in dictionary.items():
		# Accounts for multiplicities
		bow[j:j+value] = allfeatnames[allfeatnames == key].index.item()
		j += value
	# Occasional progress check
	# Note: Remaining index values are not uniformly distributed
	if not index%1000:
		print(f"About {index/5687702:.2f}% done")
	return bow

# bag_size should be at least this (set one higher just to be sure)
print("bag_size: ", allfeats.Features.apply(lambda x: sum(x.values())).max())

# Set column same as index and apply id2bow
allfeats["Bag_of_Words"] = allfeats.index
allfeats.loc[:, 'Bag_of_Words'] = allfeats.Bag_of_Words.apply(id2bow)
allfeats


bag_size:  194
About 2.48% done
About 2.48% done
About 2.49% done
About 2.51% done
About 2.59% done
About 2.66% done
About 2.70% done
About 2.75% done
About 2.78% done
About 2.80% done
About 3.01% done
About 3.02% done
About 3.03% done
About 3.28% done
About 3.30% done
About 3.37% done
About 3.53% done
About 3.66% done
About 3.77% done
About 3.91% done
About 4.31% done
About 4.32% done
About 4.34% done
About 4.58% done
About 4.65% done
About 4.80% done
About 4.80% done
About 4.80% done
About 5.63% done
About 6.02% done
About 6.95% done
About 6.96% done
About 7.76% done
About 8.04% done
About 8.38% done
About 9.75% done
About 10.26% done
About 10.67% done
About 12.20% done
About 12.97% done
About 13.15% done
About 13.39% done
About 14.10% done
About 16.38% done
About 17.40% done
About 18.14% done
About 22.18% done
About 22.23% done
About 22.46% done
About 23.08% done
About 23.96% done
About 29.03% done
About 34.05% done
About 34.59% done
About 34.84% done
About 35.67% done
About 37.26% 

C:\Users\Krisztián\AppData\Local\Temp\ipykernel_14860\769440309.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[tensor([ 31517,     86, 115673,   6741,   6741,  25306,   7622,  25768,  13811,
         115629,   7120,   1681,  45262,  24086,   8859,   7310,  45288,  45228,
          25336,  25336,  31027,   2976,   2976,  19157,  88730,  24332,  38647,
          25698,  51775,  59974,  60065,  60010,  60015,  39992,   3173,   3122,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
 

,Features,Bag_of_Words
12,"{'claychristensen': 1, 'coachella': 1, 'gabrie...","[tensor(31517), tensor(86), tensor(115673), te..."
13,"{'brainpicker': 1, 'ev': 1, 'eventbrite': 1, '...","[tensor(3462), tensor(11073), tensor(24296), t..."
17,"{'amyquispe': 1, 'baratunde': 1, 'busterbenson...","[tensor(70048), tensor(6685), tensor(24196), t..."
20,"{'aaronsw': 1, 'abdur': 1, 'amac': 1, 'dustin'...","[tensor(24104), tensor(51836), tensor(25369), ..."
47,"{'al3x': 1, 'alexandrak': 1, 'allspaw': 1, 'av...","[tensor(25362), tensor(25367), tensor(25368), ..."
...,...,...
568552194,"{'a_olivieri11': 1, 'atlanta_falcons': 1, 'dak...","[tensor(148053), tensor(130385), tensor(148073..."
568627575,"{'lucasgomezbr': 1, 'natystadler': 1}","[tensor(67889), tensor(67751), tensor(0), tens..."
568655523,"{'dailyprofet': 1, 'paixaojovem': 1}","[tensor(14243), tensor(56482), tensor(0), tens..."
568699879,"{'hotmail.com': 1, 'lualone': 1, 'mileybrcom':...","[tensor(676), tensor(306), tensor(318), tensor..."


In [18]:
# Save
allfeats.to_pickle("feature_bow.pkl")
allfeats


,Features,Bag_of_Words
12,"{'claychristensen': 1, 'coachella': 1, 'gabrie...","[tensor(31517), tensor(86), tensor(115673), te..."
13,"{'brainpicker': 1, 'ev': 1, 'eventbrite': 1, '...","[tensor(3462), tensor(11073), tensor(24296), t..."
17,"{'amyquispe': 1, 'baratunde': 1, 'busterbenson...","[tensor(70048), tensor(6685), tensor(24196), t..."
20,"{'aaronsw': 1, 'abdur': 1, 'amac': 1, 'dustin'...","[tensor(24104), tensor(51836), tensor(25369), ..."
47,"{'al3x': 1, 'alexandrak': 1, 'allspaw': 1, 'av...","[tensor(25362), tensor(25367), tensor(25368), ..."
...,...,...
568552194,"{'a_olivieri11': 1, 'atlanta_falcons': 1, 'dak...","[tensor(148053), tensor(130385), tensor(148073..."
568627575,"{'lucasgomezbr': 1, 'natystadler': 1}","[tensor(67889), tensor(67751), tensor(0), tens..."
568655523,"{'dailyprofet': 1, 'paixaojovem': 1}","[tensor(14243), tensor(56482), tensor(0), tens..."
568699879,"{'hotmail.com': 1, 'lualone': 1, 'mileybrcom':...","[tensor(676), tensor(306), tensor(318), tensor..."


### Loading, creating and splitting the data into dataloaders, defining train and test functions


In [ ]:
# Load files created previously

#allfeatnames = pd.read_json("feature_names.json", typ="series", dtype=str)
#print(allfeatnames.head())

# Number of different feature names for the embedding size
#emb_size = len(allfeatnames)+1
emb_size = 155522

allfeats = pd.read_pickle("feature_bow.pkl")
allfeats.head()


,Features,Bag_of_Words
12,"{'claychristensen': 1, 'coachella': 1, 'gabrie...","[tensor(31517), tensor(86), tensor(115673), te..."
13,"{'brainpicker': 1, 'ev': 1, 'eventbrite': 1, '...","[tensor(3462), tensor(11073), tensor(24296), t..."
17,"{'amyquispe': 1, 'baratunde': 1, 'busterbenson...","[tensor(70048), tensor(6685), tensor(24196), t..."
20,"{'aaronsw': 1, 'abdur': 1, 'amac': 1, 'dustin'...","[tensor(24104), tensor(51836), tensor(25369), ..."
47,"{'al3x': 1, 'alexandrak': 1, 'allspaw': 1, 'av...","[tensor(25362), tensor(25367), tensor(25368), ..."


In [3]:
# Create graph: from pandas to NetworkX to torch_geometric data object
# Complete edge list
edge_list = pd.read_csv("./Data/twitter_combined.txt", sep=" ", header=None)
print("Edge list:\n", edge_list.head())

# Complete graph
G = nx.from_pandas_edgelist(edge_list, 0, 1, create_using=nx.DiGraph)
print("Complete graph nodes and edges: ", G.number_of_nodes(), G.number_of_edges())

# Restrict to nonzero feature nodes
G = G.subgraph(allfeats.index)
print("Restricted graph nodes and edges: ", G.number_of_nodes(), G.number_of_edges())

# Give each node its it's bag of words as attribute
nx.set_node_attributes(G, {idx: allfeats.Bag_of_Words[idx].to(DEVICE) for idx in allfeats.index}, "BoW")

# Example
print("Features of node index 12:\n", G.nodes[12])

# Create the graph with a bag of words as the only node attribute
graph = from_networkx(G)
graph


Edge list:
            0          1
0  214328887   34428380
1   17116707   28465635
2  380580781   18996905
3  221036078  153460275
4  107830991   17868918
Complete graph nodes and edges:  81306 1768149
Restricted graph nodes and edges:  70923 1655299
Features of node index 12:
 {'BoW': tensor([ 31517,     86, 115673,   6741,   6741,  25306,   7622,  25768,  13811,
        115629,   7120,   1681,  45262,  24086,   8859,   7310,  45288,  45228,
         25336,  25336,  31027,   2976,   2976,  19157,  88730,  24332,  38647,
         25698,  51775,  59974,  60065,  60010,  60015,  39992,   3173,   3122,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0, 

Data(edge_index=[2, 1655299], BoW=[70923, 195], num_nodes=70923)

In [4]:
# Split the data: Apply train-val-test masks and create dataloaders

transform = RandomLinkSplit(num_val=0.1, num_test=0.1)
train_data, val_data, test_data = transform(graph)

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 20],
    neg_sampling=NegativeSampling("binary"),
    batch_size=1024,
    shuffle=True,
)

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 20],
    neg_sampling=NegativeSampling("binary"),
    batch_size=2048,
    shuffle=False,
)

test_loader = LinkNeighborLoader(
    data=test_data,
    num_neighbors=[20, 20],
    neg_sampling=NegativeSampling("binary"),
    batch_size=2048,
    shuffle=False,
)


In [29]:
# Train and test functions

# Display progress
def progress(value, max=100):
#	if value==max:
#		return ""
	return HTML(f"""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """)

def train(epoch, model):
	model.train()
	
	# Reset progress
	bar.update(progress(0, len(train_loader)))

	
	epoch_loss = 0.0
	correct = 0
	total = 0

	for i, batch in enumerate(train_loader, 0):
		# Predict if user1 follows user2
		# .edge_label_index -> edges to predict
		# .edge_label -> 0-1 edge labels of edge_label_index

		optimizer.zero_grad()
		outputs = model(batch.to(DEVICE))
		loss = criterion(outputs, batch.edge_label.to(DEVICE))
		loss.backward()
		optimizer.step()

		correct += (outputs.round()==batch.edge_label).sum().item()
		total += len(outputs)

		epoch_loss += loss.item()

		bar.update(progress(i+1, len(train_loader)))
	
	epoch_loss /= len(train_loader)
	epoch_acc = correct/total		
	return epoch_loss, epoch_acc


def eval(epoch, model, dataloader=val_loader):
	model.eval()

	# Reset progress
	bar.update(progress(0, len(dataloader)))

	epoch_loss = 0.0
	correct = 0
	total = 0

	for i, batch in enumerate(dataloader, 0):
		with torch.no_grad():
			outputs = model(batch.to(DEVICE))
			loss = criterion(outputs, batch.edge_label.to(DEVICE))
			correct += (outputs.round()==batch.edge_label).sum().item()
			total += len(outputs)

		epoch_loss += loss.item()

		bar.update(progress(i+1, len(dataloader)))

	
	epoch_loss /= len(dataloader)
	epoch_acc = correct/total
	return epoch_loss, epoch_acc


### Creating a node2vec embedding of the graph nodes

In [ ]:
# No need to run this block if we don't want to change the hyperparameters

def n2v_train(numEpoch = 5,
			  batch_size = 128,
			  lr = 0.001,
			  **kwargs
			  ):
	n2v = gnn.Node2Vec(**kwargs).to(DEVICE)
	loader = n2v.loader(batch_size=batch_size, shuffle=True)
	optimizer = optim.Adam(n2v.parameters(), lr=lr)

	n2v.train()
	for epoch in range(numEpoch):
		epoch_loss = 0
		for pos_rw, neg_rw in loader:
			optimizer.zero_grad()
			loss = n2v.loss(pos_rw.to(DEVICE), neg_rw.to(DEVICE))
			loss.backward()
			optimizer.step()
			epoch_loss += loss.item()
		print(f"Epoch {epoch+1} loss: {epoch_loss/len(loader):.4f}")
	return n2v

node2vec_trained = n2v_train(numEpoch=100,
							 batch_size=512,
							 edge_index = train_data.edge_index,
							 embedding_dim = 20,
							 walk_length = 10,
							 context_size = 10,
							 walks_per_node = 500,
							 )

torch.save(node2vec_trained.state_dict(), "node2vec_trained.pt")


In [6]:
# Load node2vec embedding
state = torch.load("node2vec_trained.pt")
n2v = gnn.Node2Vec(train_data.edge_index, 20, 10, 10).to(DEVICE)
n2v.load_state_dict(state)


C:\Users\Krisztián\AppData\Local\Temp\ipykernel_18828\1831015820.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load("node2vec_trained.pt")


<All keys matched successfully>

## Baseline models

Not using the graph structure

In [ ]:
# Matrix factorization
# Theory: People tend to follow people similar to them (i.e. form echo chambers)
# Problem: Symmetric, which is not accurate for celebrities, for example (people with many followers)
class MatrixFactorization(nn.Module):
    def __init__(self, num_features = emb_size, embedding_dim = 20):
        super().__init__()
        self.emb = nn.EmbeddingBag(num_features,
                                   embedding_dim,
                                   max_norm=1,
                                   scale_grad_by_freq=True,
                                   mode="sum",
                                   padding_idx=0,
                                   )
    
    def forward(self, data):
        edge_index = data.edge_label_index
        user1_feat, user2_feat = data.BoW[edge_index]

        user1_vector = self.emb(user1_feat)
        user2_vector = self.emb(user2_feat)
        return (user1_vector * user2_vector).sum(1)+0.5


In [30]:
lr, emb_dim = 0.001, 20
model = MatrixFactorization(emb_size, emb_dim).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.BCEWithLogitsLoss().to(DEVICE)
numEpoch = 5

t_losses = []
t_accs = []
v_losses = []
v_accs = []

best_loss = 100.0
best_acc = 0.0
# Progress bar
bar = display(progress(0, len(train_loader)), display_id=True)
for epoch in range(numEpoch):
	
	# Train
	t_loss, t_acc = train(epoch, model)
	t_losses.append(t_loss)
	t_accs.append(t_acc)
	print(f"Epoch {epoch+1}: Train loss: {t_loss:.4f}, - accuracy: {t_acc:.4f}")

	# Validate
	v_loss, v_acc = eval(epoch, model)
	v_losses.append(v_loss)
	v_accs.append(v_acc)
	print(F"Epoch {epoch+1}: Valid loss: {v_loss:.4f} - accuracy: {v_acc:.4f}")

	# LR scheduler
	# scheduler.step()

	# Compare to current best accuracy
	if v_loss <= best_loss:
		best_loss = v_loss
		best_acc = v_acc

print(f"Best val_acc: {best_acc}")


Epoch 1: Train loss: 1.1404, - accuracy: 0.1498
Epoch 1: Valid loss: 0.8558 - accuracy: 0.1734
Epoch 2: Train loss: 0.7817, - accuracy: 0.1925
Epoch 2: Valid loss: 0.7146 - accuracy: 0.2065
Epoch 3: Train loss: 0.6839, - accuracy: 0.2170
Epoch 3: Valid loss: 0.6513 - accuracy: 0.2228
Epoch 4: Train loss: 0.6364, - accuracy: 0.2288
Epoch 4: Valid loss: 0.6198 - accuracy: 0.2301
Epoch 5: Train loss: 0.6120, - accuracy: 0.2345
Epoch 5: Valid loss: 0.6017 - accuracy: 0.2337
Best val_acc: 0.2337365328516486


In [33]:
# Two FC Layers
class Linear(nn.Module):
	def __init__(self, num_features = emb_size, embedding_dim = 20, hidden_dim = 32):
		super().__init__()
		self.emb = nn.EmbeddingBag(num_features, embedding_dim, max_norm=1, scale_grad_by_freq=True, padding_idx=0)

		self.lin1 = nn.Linear(2*embedding_dim, hidden_dim)
		#self.lin1 = nn.Linear(embedding_dim, hidden_dim)
		self.lin2 = nn.Linear(hidden_dim, 1)
		self.lin2.bias = nn.Parameter(torch.full(self.lin2.bias.shape, 0.5))
		self.nonlin = nn.ReLU()
		self.drop = nn.Dropout(0.5)

	def forward(self, data):
		edge_index = data.edge_label_index
		user1_feat, user2_feat = data.BoW[edge_index]

		user1_vector = self.emb(user1_feat)
		user2_vector = self.emb(user2_feat)

		x = torch.cat([user1_vector, user2_vector], dim=1)
		#x = user1_vector + user2_vector
		
		out = self.lin2(self.drop(self.nonlin(self.lin1(x)))).squeeze()
		return out


In [34]:
lr, emb_dim, hidden_dim = 0.1, 20, 256
model = Linear(emb_size, emb_dim, hidden_dim).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = torch.nn.BCEWithLogitsLoss().to(DEVICE)
numEpoch = 5

t_losses = []
t_accs = []
v_losses = []
v_accs = []

best_loss = 100.0
best_acc = 0.0
# Progress bar
bar = display(progress(0, len(train_loader)), display_id=True)
for epoch in range(numEpoch):
	# Train
	t_loss, t_acc = train(epoch, model)
	t_losses.append(t_loss)
	t_accs.append(t_acc)
	print(f"Epoch {epoch+1}: Train loss: {t_loss:.4f}, - accuracy: {t_acc:.4f}")

	# Validate
	v_loss, v_acc = eval(epoch, model)
	v_losses.append(v_loss)
	v_accs.append(v_acc)
	print(F"Epoch {epoch+1}: Valid loss: {v_loss:.4f} - accuracy: {v_acc:.4f}")

	# LR scheduler
	# scheduler.step()

	# Compare to current best accuracy
	if v_loss <= best_loss:
		best_loss = v_loss
		best_acc = v_acc

print(f"Best val_acc: {best_acc}")


Epoch 1: Train loss: 0.6715, - accuracy: 0.4811
Epoch 1: Valid loss: 0.6505 - accuracy: 0.4376
Epoch 2: Train loss: 0.6386, - accuracy: 0.4170
Epoch 2: Valid loss: 0.6267 - accuracy: 0.4318
Epoch 3: Train loss: 0.6233, - accuracy: 0.4583
Epoch 3: Valid loss: 0.6153 - accuracy: 0.4777
Epoch 4: Train loss: 0.6160, - accuracy: 0.4812
Epoch 4: Valid loss: 0.6096 - accuracy: 0.4805
Epoch 5: Train loss: 0.6119, - accuracy: 0.4865
Epoch 5: Valid loss: 0.6058 - accuracy: 0.4934
Best val_acc: 0.4933773384149864


## Basic GNN model

In [ ]:
# Two CGN layers with reverse message passing, because the people we follow influence us
# (The reverse is also true, but to a lesser degree)
class GCN(torch.nn.Module):
	def __init__(self, num_features=emb_size, embedding_dim = 20, hidden_dim = 32):
		super().__init__()
		self.emb = nn.EmbeddingBag(num_features,
                                   embedding_dim,
                                   max_norm=1,
                                   scale_grad_by_freq=True,
                                   mode="sum",
                                   padding_idx=0,
                                   )
		self.n2v = n2v
		self.combine = nn.Linear(2*embedding_dim, embedding_dim)

		self.conv1 = gnn.GCNConv(n2v.embedding_dim, hidden_dim, flow="target_to_source")
		self.conv2 = gnn.GCNConv(hidden_dim, hidden_dim, flow="target_to_source")

		self.nonlin = nn.ReLU()
		self.drop = nn.Dropout(0.5)
		
		self.classifier = gnn.Linear(hidden_dim, 1)
		#self.classifier.bias = nn.Parameter(torch.full(self.classifier.bias.shape, 0.5))

	def forward(self, data):
		node_index, node_bow, edge_index, label_index = data.n_id, data.BoW, data.edge_index, data.edge_label_index
		
		# Combine node embeddings of the bag of words and node2vec
		x1 = self.emb(node_bow)
		x2 = self.n2v(node_index)
		x = torch.cat([x1,x2], dim=1)
		x = self.combine(self.nonlin(x))

		# cuda dies here...
		out = self.conv1(x, edge_index)
		out = self.conv2(self.drop(self.nonlin(out)), edge_index)

		out = self.classifier(self.drop(self.nonlin(out))).squeeze()
		out = out[label_index[0]]
		return out


In [38]:
lr, emb_dim, hidden = 0.001, 20, 256
model = GCN(emb_size, emb_dim, hidden).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.BCEWithLogitsLoss().to(DEVICE)
numEpoch = 5

t_losses = []
t_accs = []
v_losses = []
v_accs = []

best_loss = 1000.0
best_acc = 0.0
# Progress bar
bar = display(progress(0, len(train_loader)), display_id=True)
for epoch in range(numEpoch):
	# Train
	t_loss, t_acc = train(epoch, model)
	t_losses.append(t_loss)
	t_accs.append(t_acc)
	print(f"Epoch {epoch+1}: Train loss: {t_loss:.4f}, - accuracy: {t_acc:.4f}")

	# Validate
	v_loss, v_acc = eval(epoch, model)
	v_losses.append(v_loss)
	v_accs.append(v_acc)
	print(F"Epoch {epoch+1}: Valid loss: {v_loss:.4f} - accuracy: {v_acc:.4f}")

	# LR scheduler
	# scheduler.step()

	# Compare to current best accuracy
	if v_loss <= best_loss:
		best_loss = v_loss
		best_acc = v_acc

print(f"Best val_acc: {best_acc}")


torch.Size([48767, 20])
cuda:0
torch.Size([48988, 20])
cuda:0
torch.Size([48839, 20])
cuda:0
torch.Size([48878, 20])
cuda:0
torch.Size([48954, 20])
cuda:0
torch.Size([49021, 20])
cuda:0
torch.Size([49179, 20])
cuda:0
torch.Size([48470, 20])
cuda:0
torch.Size([49141, 20])
cuda:0
torch.Size([48766, 20])
cuda:0
torch.Size([48501, 20])
cuda:0
torch.Size([48925, 20])
cuda:0
torch.Size([48442, 20])
cuda:0
torch.Size([48687, 20])
cuda:0
torch.Size([48920, 20])
cuda:0
torch.Size([48560, 20])
cuda:0
torch.Size([49096, 20])
cuda:0
torch.Size([49063, 20])
cuda:0
torch.Size([49121, 20])
cuda:0
torch.Size([48731, 20])
cuda:0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Advanced GNN model

In [ ]:
# TODO: Something that actually works


#### Obsolete code

This was my original plan before settling on bag-of-words embeddings of the node features

(Note: This would have resulted in ~71k (sparse) tensor files, which are not large, but slow to look up, and the graph would need too much memory anyways)


In [39]:
# Dictionary to tensor for function saving individual vectors
def dict2vec(d, idx):
	vector = torch.zeros(len(allfeatnames)+1, dtype=torch.int64)
	for k,v in d.items():
		vector[allfeatnames.loc[allfeatnames == k].index] = v
	torch.save(vector.to_sparse(), f"./Data/tensors/vector{idx}.pt")

# Helper function to load feature vectors by node ID
def id2vec(idx):
	if os.path.isfile(f"./Data/tensors/vector{idx}.pt"):
		return torch.load(f"./Data/tensors/vector{idx}.pt").to_dense().to(DEVICE)
	print("Bad index")
	return torch.zeros(emb_size).to(DEVICE)

# Huge computation, never run this again!
#for idx in allfeats.index:
#	dict2vec(allfeats.loc[idx].item(), idx)
